# Paquetes

In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import tldextract
import jellyfish
from joblib import dump, load

# Importar Dataset

Luego de haber tuneado y evaluado los modelos con el Train set, los elegimos según su performance en el Test set. Elegido el mejor de los modelos con el set de Test, entrenamos dicho modelo elegido con TODOS los datos. Acá importamos el dataset completo.

In [ ]:
# Importar dataset completo

# Entrenar Modelo

En esta parte entrenamos al modelo elegido y guardamos el objeto en un archivo **joblib**, para luego ser importado por el script de la aplicación y poder ser utilizado de manera directa en las predicciones.

In [12]:
# Entrenamos el modelo con el dataset completo.

In [11]:
# Exportamos el archivo con el modelo entrenado.
dump(model, 'trained_model.joblib') 

In [5]:
rf = load('/home/jcc/JuanCruz_Ubuntu/Repositorios_Activos/Proyecto_CDD/Proyecto A/00 - Creación Dataset y Variables/trained_rf.joblib')

In [9]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

# Prueba Aplicación

Podemos importar el módulo creado para usar la aplicación en un set de URLs que querramos.

In [2]:
urls = ['https://git-scm.com/docs/gitignore', 
        'https://scikit-learn.org/stable/modules/model_persistence.html',
        'https://www.lanacion.com.ar/',
        'https://articulo.mercadolibre.com.ar/MLA-737102690-sommier-king-la-cardeuse-native-400-200x200-_JM?variation=95000501490#reco_item_pos=2&reco_backend=promotions-sorted-by-score-mla-A&reco_backend_type=low_level&reco_client=seller-promotions&reco_id=574ab7bd-21ab-43a8-a868-69b5785b0fc9&deal_print_id=25c97840-04f2-11ec-85d9-336a60e0da68&c_id=carouseldynamic-home&c_element_order=undefined&c_campaign=OFERTAS-X-24-HORAS&c_uid=25c97840-04f2-11ec-85d9-336a60e0da68',
        'https://www.clarin.com/']

prueba = pd.DataFrame({'url':urls})

In [3]:
from Aplicacion import DetectarPhishing

In [4]:
detector = DetectarPhishing()

Instancia del detector fue creada


In [44]:
pd.get_dummies(prueba['suffix']=='').columns[0] == False

KeyError: 'suffix'

In [10]:
metric_domains = [
    'amazon', 
    'instagram', 
    'google', 
    'whatsapp',  
    'twitter',
    'facebook',
    'yahoo', 
    'wikipedia',
    'baidu',
    'paypal', 
    'mail', 
    'sfexpress' ,
    'onedrive',
    'excel', 
    'square', 
    'mail', 
    'office365', 
    'irs', 
    'tencent', 
    'creditagrecole s.a.',
    'microsoft',
    'blogspot',
    'onedrive',
    'payment',
    'hsbc',
    'secure',
    'help',
    'banco',
    'bank',
    'support',
    'rakuten',
    'steam',
    'olx'
]

In [56]:
        data_urls = prueba
        # Descomposición del URL.

        data_urls['scheme'] = data_urls['url'].apply(lambda x: urlparse(x).scheme)

        data_urls['domain_complete'] = data_urls['url'].apply(lambda x: urlparse(x).netloc)
        data_urls['domain_complete'] = data_urls['domain_complete'].str.replace('www.', '')
        data_urls['domain_complete'] = data_urls['domain_complete'].str.replace('www', '')

        data_urls['domain'] = data_urls['domain_complete'].apply(lambda x: tldextract.extract(x).domain)
        data_urls['subdomain'] = data_urls['domain_complete'].apply(lambda x: tldextract.extract(x).subdomain)
        data_urls['suffix'] = data_urls['domain_complete'].apply(lambda x: tldextract.extract(x).suffix)

        data_urls['subdomain'] = data_urls['subdomain'] + '.'
        data_urls['subdomain'] = data_urls['subdomain'].replace('.', '')

        data_urls['domain_subdomain'] = data_urls['subdomain'] + data_urls['domain']

        data_urls['path'] = data_urls['url'].apply(lambda x: urlparse(x).path)

        # Creación de Variables.

        # Variables del dominio
        # Cuenta los puntos
        data_urls['dom_n_puntos'] = data_urls['domain_subdomain'].str.count('\\.')
        data_urls['dom_n_guion'] = data_urls['domain_subdomain'].str.count('\\-')
        data_urls['dom_n_guionbajo'] = data_urls['domain_subdomain'].str.count('\\_')

        # Cuenta el largo total del dominio + subdominio
        data_urls['dom_len_tot'] = data_urls['domain_subdomain'].str.len()
        # Cuenta el largo del dominio y subdominio por separado
        data_urls['dom_len'] = data_urls['domain'].str.len()
        data_urls['dom_len_sub'] = data_urls['subdomain'].str.len()
        data_urls['url_len'] = data_urls['url'].str.len()
        # Cuenta vocales
        data_urls['dom_vocales'] = data_urls['domain_subdomain'].str.lower().str.count(r'[aeiou]')
        # Cuenta consonantes
        data_urls['dom_cons'] = data_urls['domain_subdomain'].str.lower().str.count(r'[a-z]') - data_urls['dom_vocales']
        # Cuenta números
        data_urls['dom_num'] = data_urls['domain_subdomain'].str.count('\d')
        # Cuenta cantidad de caracteres diferentes
        data_urls['dom_car_dif'] = data_urls['domain_subdomain'].apply(set).apply(len)

        # Dominio es IP
        ip_dummies = pd.get_dummies(data_urls['suffix'] == '')
        if ip_dummies.shape[1] == 2:
            data_urls['dom_ip'] = pd.get_dummies(data_urls['suffix'] == '', drop_first=True)
        elif ip_dummies.shape[1] == 1:
            if ip_dummies.columns[0] == False:
                data_urls['dom_ip'] = pd.get_dummies(data_urls['suffix'] == '').replace(1,0)
            else:
                data_urls['dom_ip'] = pd.get_dummies(data_urls['suffix'] == '')
            
        # Variable dummy Scheme
        sch_dummies = pd.get_dummies(data_urls['scheme'], prefix='sch')
        data_urls = pd.concat([data_urls, sch_dummies], axis = 1)

        # Variables de Suffix
        data_urls['suf_len'] = data_urls['suffix'].str.len()

        # Creamos e imprimimos una lista con el top 5 de sufijos.
        top_suf_list = ['com', 'net', 'org', 'ru', 'xyz']
        data_urls['suffix2'] = data_urls['suffix']
        # Asignamos categoría 'other' a todas las clases que no pertenezcan a top_suf_list.
        data_urls.loc[data_urls['suffix2'].isin(top_suf_list).apply(np.bitwise_not), 'suffix2'] = 'other'
        # Armamos las columnas dummy.
        suf_dummies = pd.get_dummies(data_urls['suffix2'], prefix='suf')
        data_urls = pd.concat([data_urls, suf_dummies], axis = 1)

        for domain in metric_domains:
            data_urls['metric_ds_'+domain] = data_urls['domain_subdomain'].apply(lambda x: jellyfish.jaro_winkler(x, domain))
            data_urls['metric_d_'+domain] = data_urls['domain'].apply(lambda x: jellyfish.jaro_winkler(x, domain))
            data_urls['metric_s_'+domain] = data_urls['subdomain'].apply(lambda x: jellyfish.jaro_winkler(x, domain))
            data_urls['metric_p_'+domain] = data_urls['path'].apply(lambda x: jellyfish.jaro_winkler(x, domain))

In [57]:
data_urls['dom_ip']

0    0
1    0
2    0
3    0
4    0
Name: dom_ip, dtype: uint8

In [5]:
detector.prepararInput(prueba)

In [16]:
detector.data.iloc[:,9:30]

,dom_num,dom_car_dif,dom_ip,sch_https,suf_len,suf_com,suf_org,suf_other,metric_ds_amazon,metric_d_amazon,...,metric_p_amazon,metric_ds_instagram,metric_d_instagram,metric_s_instagram,metric_p_instagram,metric_ds_google,metric_d_google,metric_s_google,metric_p_google,metric_ds_whatsapp
0,0,7,0,1,3,1,0,0,0.000000,0.000000,...,0.488889,0.588624,0.588624,0.000000,0.400000,0.436508,0.436508,0.000000,0.322222,0.511905
1,0,11,0,1,3,0,1,0,0.000000,0.000000,...,0.526316,0.590741,0.590741,0.000000,0.495712,0.500000,0.500000,0.000000,0.423977,0.430556
2,0,6,0,1,6,0,0,1,0.638889,0.638889,...,0.000000,0.324074,0.324074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
3,0,13,0,1,6,0,0,1,0.436508,0.472222,...,0.504659,0.484127,0.296296,0.314815,0.547640,0.436508,0.472222,0.425926,0.410394,0.505952
4,0,6,0,1,3,1,0,0,0.555556,0.555556,...,0.000000,0.518519,0.518519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.430556


**Hay que corregir la generación de dummies, get_dummies genera con las que están presentes pero no considera las que no aparecen. Hay que hacerlas fijas armando funciones para cada una.**

In [9]:
detector.calcularProbabilidad(prueba)

ValueError: Number of features of the model must match the input. Model n_features is 145 and input n_features is 141 